<a href="https://colab.research.google.com/github/WagnerCorrey/Aula_pratica_Alura_Google/blob/main/Projeto_aula_pr%C3%A1tica_Imers%C3%A3o_Google%2BAlura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install -q -U google-generativeai

In [1]:
# @title
import textwrap
import numpy as np
import pandas as pd

import google.generativeai as genai
import google.ai.generativelanguage as glm
from google.colab import userdata
api_key = userdata.get('sigiloso')
genai.configure(api_key=api_key)

In [1]:
!pip install PyPDF2 transformers rasa sentence-transformers


!pip install rasa

In [4]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [29]:
import PyPDF2
import requests
import io
import tempfile

def extrair_texto_pdf(caminho_arquivo):
  with open(caminho_arquivo, 'rb') as pdf_file:
      pdf_reader = PyPDF2.PdfReader(pdf_file)

      texto_completo = ""
      for num_pagina in range(len(pdf_reader.pages)):
          pagina = pdf_reader.pages[num_pagina]
          texto_completo += pagina.extract_text()

  return texto_completo

# arquivo PDF usando a biblioteca requests
url_pdf = 'https://www.ipcc.ch/report/ar6/syr/downloads/report/IPCC_AR6_SYR_FullVolume.pdf'
response = requests.get(url_pdf)


with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as temp_file:
    temp_file.write(response.content)


texto_ipcc = extrair_texto_pdf(temp_file.name)

In [31]:
import requests
from bs4 import BeautifulSoup

# Fazendo uma solicitação HTTP para o site
url = 'https://science.nasa.gov/climate-change/'
response = requests.get(url)


if response.status_code == 200:
    # Criando um objeto BeautifulSoup para analisar o conteúdo HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extraindo informações específicas do site

    links = soup.find_all('a')

    for link in links:
        print(link.get('href'))
else:
    print('Falha ao fazer a solicitação HTTP')

#primary
https://www.nasa.gov/missions/
https://www.nasa.gov/a-to-z-of-nasa-missions/
https://www.nasa.gov/events/
https://www.nasa.gov/humans-in-space/spaceships-and-rockets/
https://www.nasa.gov/communicating-with-missions/
https://www.nasa.gov/humans-in-space/artemis/
https://science.nasa.gov/mission/webb/
https://science.nasa.gov/mission/hubble/
https://www.nasa.gov/international-space-station/
https://science.nasa.gov/mission/osiris-rex/
https://www.nasa.gov/humans-in-space/why-go-to-space/
https://www.nasa.gov/humans-in-space/astronauts/
https://www.nasa.gov/humans-in-space/commercial-space/
https://www.nasa.gov/humans-in-space/destinations/
https://www.nasa.gov/humans-in-space/spaceships-and-rockets/
https://www.nasa.gov/humans-in-space/living-in-space/
https://science.nasa.gov/earth/explore/
https://science.nasa.gov/climate-change/
https://science.nasa.gov/earth/facts/
https://science.nasa.gov/earth/in-action/
https://science.nasa.gov/earth/multimedia/
https://science.nasa.gov/

In [32]:
import re

def limpar_texto(texto):
  texto = re.sub(r'\s+', ' ', texto)
  texto = re.sub(r'\n+', '\n', texto)
  texto = re.sub(r'\d+\s', '', texto)
  return texto

texto_ipcc_limpo = limpar_texto(texto_ipcc)

In [36]:
import requests
from bs4 import BeautifulSoup

# Solicitação HTTP para o site
url = 'https://science.nasa.gov/climate-change/'
response = requests.get(url)


if response.status_code == 200:
    # Criando um objeto BeautifulSoup para analisar o conteúdo HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extraindo o texto dos parágrafos
    texto_paragrafos = ''
    for p in soup.find_all('p'):
        texto_paragrafos += p.text


In [38]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Modelo de embeddings
model = SentenceTransformer('all-mpnet-base-v2')


df = pd.DataFrame(columns=['tema', 'pergunta', 'resposta', 'fonte', 'embedding'])

# informações sobre mudanças climáticas (extraídas do site da NASA)
df = pd.concat([df, pd.DataFrame({'tema': 'Mudanças Climáticas', 'pergunta': 'O que é o aquecimento global?',
                'resposta': texto_paragrafos, # Texto extraído do site da NASA
                'fonte': 'NASA Climate', 'embedding': model.encode(texto_paragrafos)})], ignore_index=True)

# informações sobre HAARP
df = pd.concat([df, pd.DataFrame({'tema': 'Teorias da Conspiração', 'pergunta': 'O HAARP controla o clima?',
                'resposta': 'O HAARP é um programa de pesquisa da ionosfera e não tem capacidade de controlar o clima. Essa teoria da conspiração não possui base científica.',
                'fonte': 'Skeptical Inquirer', 'embedding': model.encode('O HAARP é um programa de pesquisa da ionosfera e não tem capacidade de controlar o clima. Essa teoria da conspiração não possui base científica.')})], ignore_index=True)

# informações sobre Blue Beam
df = pd.concat([df, pd.DataFrame({'tema': ['Teorias da Conspiração'], 'pergunta': ['A teoria do Blue Beam é real?'],
                'resposta': ['A teoria do Blue Beam, que alega um\n        \n'],
                'fonte': ['Skeptical Inquirer'], 'embedding': [model.encode('A teoria do Blue Beam, que alega um\n        \n')]})])

In [ ]:
# Crie um projeto Rasa
!rasa init --no-prompt


!rasa train

!rasa run actions &
!rasa shell

# **Se fosse usado como integrado a uma conta no Instagram**

**Código usado:**

!pip install instabot

from instabot import Bot

bot = Bot()
bot.login(username='seu_usuario', password='sua_senha')

@bot.on_message()
def responder_mensagem(message):
  
  bot.send_message(resposta, message['user_id'])

bot.run()